In [121]:
#!pip install selenium

In [2]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import string

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
driver=Chrome()

In [5]:
url='https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/'

In [6]:
driver.get(url)
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="08bbef08ac43171ddfc9731761620d57")>


In [7]:
driver.maximize_window()

In [8]:
shadow_host = driver.find_element(By.CSS_SELECTOR, "[id='usercentrics-root']")
time.sleep(3)
button = shadow_host.shadow_root.find_element(By.CSS_SELECTOR,"[data-testid='uc-accept-all-button']")
button.click()

In [9]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [10]:
titletxt=soup.title.text
print(titletxt)

exhibitor list - The World of ITB


In [11]:
alpha=list(string.ascii_uppercase)
alpha.append('#')
print(alpha)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '#']


In [26]:
res=[] 
anchors=[]
i=0
instagram_pattern = re.compile(r"(?:http(?:s)?:\/\/)?(?:www\.)?instagram\.com\/[a-zA-Z0-9_]+\/?")
facebook_pattern = re.compile(r"(?:http(?:s)?:\/\/)?(?:www\.)?facebook\.com\/[a-zA-Z0-9_]+\/?")
linkedin_pattern = re.compile(r"(?:http(?:s)?:\/\/)?(?:www\.)?linkedin\.com\/[a-zA-Z0-9_]+\/?")

instagram_links = []
facebook_links = []
linkedin_links = []
alpha='T'
for a in alpha:
    url = 'https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/search/f=h-entity_orga;alpha='+a+';v_sg=0;v_fg=0;v_fpa=FUTURE'
    driver.get(url)
    time.sleep(3)
    try:
        while True:
            show_more_button = soup.find('div', class_='EWP5KKC-u-a EWP5KKC-u-d')
            if show_more_button:
                show_more_button_element = driver.find_element(By.XPATH,'//*[@id="onlineGuide"]/div/div[4]/div[2]/div[2]/div[4]/div[1]/div')
                driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button_element)
                show_more_button_element.click()
                time.sleep(2)
            else:
                print(f"No more pages.")
                break
    except Exception as e:
        print(f"Error: {e}")
    
    time.sleep(10)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    exhibitors_details=soup.find_all('div', class_="listContentContainer")
    
    anchor_elements=driver.find_elements(By.CSS_SELECTOR,"a[class='gwt-Anchor EWP5KKC-d-m']")
    anchors=[]
    
    for i in range(len(anchor_elements)):
           anchors.append(anchor_elements[i].text.replace(' ','').strip())
        
    for ex in exhibitors_details:
        section=ex.find_all('div', class_="EWP5KKC-w-l")
        for e in section:
            exhibitor=e.find('div', class_='gwt-Label EWP5KKC-w-Q').text.replace(' ','')
            address= e.find('div', class_="gwt-Label EWP5KKC-w-m").text
        
            Desc=e.find('div', class_='gwt-HTML EWP5KKC-w-u')
            Desc = Desc.text if Desc else "N/A"
        
            print(f"\nExhibitor Name: {exhibitor.strip()}")   
            print(f"Address: {address.strip()}")
            print(f"Description: {Desc.strip()}")
        
            if exhibitor in anchors:        
                index = anchors.index(exhibitor)
                href_link = anchor_elements[index].get_attribute('href')
                print("href link:", href_link)
                driver.execute_script("arguments[0].scrollIntoView({'block': 'center', 'inline': 'nearest', 'behaviour': 'instant'});", anchor_elements[index]);
                anchor_elements[index].click() 
                time.sleep(2)
                i=i+1
                new_soup = BeautifulSoup(driver.page_source, "html.parser")
                parent = new_soup.find('div',id="onlineGuidePopup")
                links = parent.find_all("a")
                Ilink=""
                Flink=""
                Llink=""
                for link in links:
                    shref = link.get("href")
                    if shref:
                        if re.search(instagram_pattern, shref):
                            instagram_links.append(shref)
                            Ilink=shref
                        elif re.search(facebook_pattern, shref):
                            facebook_links.append(shref)
                            Flink=shref
                        elif re.search(linkedin_pattern, shref):
                            linkedin_links.append(shref)
                            Llink=shref
                         
                                                 
                row=[exhibitor.strip(),address.strip(),Desc.strip(),href_link,Ilink,Flink,Llink] 
                res.append(row)

                closeIcon = driver.find_element(By.CSS_SELECTOR,"[class='xIcon EWP5KKC-y-l']")
                closeIcon.click()
    
            
df = pd.DataFrame(res, columns=["Name", "Address", "Description", "More_info","Instagram","Facebook","LinkedIn"])             
df.to_csv(r'C:\Users\harsh\Python Practice\Web Scraping\ITB - Exhibitors.csv', index=False)    

Error: Message: element not interactable
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF63472AD22+56930]
	(No symbol) [0x00007FF63469F622]
	(No symbol) [0x00007FF63455419D]
	(No symbol) [0x00007FF63459ACA2]
	(No symbol) [0x00007FF63458F798]
	(No symbol) [0x00007FF6345BBC9A]
	(No symbol) [0x00007FF63458F09A]
	(No symbol) [0x00007FF6345BBEB0]
	(No symbol) [0x00007FF6345D81E2]
	(No symbol) [0x00007FF6345BBA43]
	(No symbol) [0x00007FF63458D438]
	(No symbol) [0x00007FF63458E4D1]
	GetHandleVerifier [0x00007FF634AA6AAD+3709933]
	GetHandleVerifier [0x00007FF634AFFFED+4075821]
	GetHandleVerifier [0x00007FF634AF817F+4043455]
	GetHandleVerifier [0x00007FF6347C9756+706710]
	(No symbol) [0x00007FF6346AB8FF]
	(No symbol) [0x00007FF6346A6AE4]
	(No symbol) [0x00007FF6346A6C3C]
	(No symbol) [0x00007FF6346968F4]
	BaseThreadInitThunk [0x00007FFCBDC07344+20]
	RtlUserThreadStart [0x00007FFCBEF426B1+33]


Exhibitor Name: ta.ts-TravelAgencyTechnologies&ServiceGmbH
Address: 60


Exhibitor Name: TandemTravelDMCBulgaria
Address: 1124 Sofia, Bulgaria
Description: Why to choose Tandem Travel? BECAUSE WE USE OUR EXPERIENCE TO CREATE YOURS!
We are a friendly and  innovative team with over 25 years experience. The company is based in Sofia, Bulgaria and offers high quality services and unforgettable local experiences with respect to  responsible travel.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/tandem-travel-dmc-bulgaria--1039478

Exhibitor Name: TANGANYIKAEXPEDITIONS
Address: 0 Arusha, Tanzania
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/tanganyika-expeditions--1207049

Exhibitor Name: TanganyikaWildernessCamps
Address: 000 Arusha, Tanzania
Description: Tanganyika Wilderness Camps owns and manages a collection of 18 boutique style safari properties, including Tented Lodges, Mobile Camps, all of which are located in the prime areas of northern Tanzania, such as: West Kilimanj


Exhibitor Name: TaylorsSafaris
Address: . Bulawayo, Zimbabwe
Description: Taylors Safaris = Tailor Made Safaris in Zimbabwe
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/taylors-safaris--1206473

Exhibitor Name: TazabayGrandServiceLLC
Address: 230100 Nukus, Uzbekistan
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/tazabay-grand-service-llc--1205596

Exhibitor Name: TA’AKTANA,aLuxuryCollectionResort&Spa,LabuanBajo
Address: 86763 Labuan Bajo, Indonesia
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/ta-aktana-a-luxury-collection-resort-spa-labuan-bajo--1206187

Exhibitor Name: TbilisiCityHall
Address: 0160 Tbilisi, Georgia
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/tbilisi-city-hall--1039600

Exhibitor Name: TbilisiPhilharmonicandSaburtalobyMercurehotels
Address: 0179 Tbilisi, Georgia
Description: Tbilis


Exhibitor Name: TerraNovaAustralia
Address: 2060 North Sydney, Australia
Description: With over 30 years of experience, Terra Nova Tours specialises in group tours, tour series and guided tour programmes for individuals in New Zealand and Austral
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/terra-nova-australia--1205323

Exhibitor Name: TerraNovaToursLimited&TheGreenSpotLimited
Address: 8013 Christchurch, New Zealand
Description: With over 30 years of experience, Terra Nova Tours specialises in group tours, tour series and guided tour programmes for individuals in New Zealand and Austral
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/terra-nova-tours-limited-the-green-spot-limited--1039940

Exhibitor Name: TerraNovaTours
Address: 8002 Waterfront, South Africa
Description: Terra Nova is a well-known DMC with offices in South Africa and Namibia. We offer excellent rates and tailormade itineraries for FIT's and group


Exhibitor Name: THBHOTELS
Address: 07458 Palma de Mallorca, Spain
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/thb-hotels--1040090

Exhibitor Name: TheAbuDhabiEDITION
Address: 127799 Abu Dhabi, United Arab Emirates
Description: EDITION Hotels combine a personal, intimate, individualized and one-of-a-kind hotel experience with the global reach and scale of Marriott International.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/the-abu-dhabi-edition--1040091

Exhibitor Name: TheAlbatrozHotel
Address: 2750-353 Cascais, Portugal
Description: The Albatroz Hotel a combination of luxury hotel service, located in the hart of Cascais
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/the-albatroz-hotel--1040098

Exhibitor Name: TheAnamResorts
Address: 57600/77118 Cam Ranh, Vietnam
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/the-

...


Exhibitor Name: TrailBlazerToursIndiaPrivateLimited
Address: 122002 Gurugram (Delhi NCR), India
Description: TBi a young and dynamic Incoming Agency specializes in all categories of voyages such as Culture & Heritage, Art & Architecture, Wildlife, Luxury, Wellness, Honeymoon, Leisure, Yoga, Culinary, Adventure, special interest tours & more. Meet Mr. Malayanil Singh, Senior Vice President at our stand.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/trail-blazer-tours-india-private-limited--1041421

Exhibitor Name: TrailsofAfricaToursandSafarisLtd
Address: 00100 Nairobi, Kenya, Kenya
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/trails-of-africa-tours-and-safaris-ltd--1208354

Exhibitor Name: TrailsOfIndochina
Address: 700000 Ho Chi Minh, Vietnam
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/trails-of-indochina--1041426

Exhibitor Name: TRANSINDIAHOLIDA


Exhibitor Name: TravelEscapesMaldives
Address: 20081 Male', Maldives
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/travel-escapes-maldives--1041558

Exhibitor Name: TravelEurope
Address: 6135 Stans, Austria
Description: Your 360° tour operator with 15 local branches throughout Europe - Travel Europe covers a wide range of packages: round trips, cruises, hotel accommodation, hiking tours, city or weekend trips, …with a great deal of attention to a contemporary product portfolio.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/travel-europe--1041560

Exhibitor Name: TravelExcellence
Address: 12468-1000 San José, Costa Rica
Description: Since 1996, Travel Excellence offers ingenious and innovative ideas to allow the maximum enjoyment of this rich country. As ground operators, we offer all the s
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/travel-excellence--1041562

Ex


Exhibitor Name: Travelport
Address: SL3 8AG Slough, United Kingdom
Description: We've got a lot to catch up with you on. Like how we're driving modern travel retailing with our Travelport+ platform, our transformative Smartpoint Cloud interface, and our Deem integration.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/travelport--1041660

Exhibitor Name: TravelProjekt
Address: 03-808 Warszawa, Poland
Description: Travel Projekt is a leading Polish group and incentive operator with 25 years of experience and precious “know-how
We offer different kind of tours and programmes for leisure and special interest groups as well as high quality , incentive and team building programs.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/travel-projekt--1207444

Exhibitor Name: TravelPromotersLtd
Address: Rh20 3HD Pulborough, United Kingdom
Description: Meet Ailsa & Leanne on the ATTA stand to hear the about the incredible additions 


Exhibitor Name: TripleATechnologiesPte.Ltd.
Address: 068914 Singapore, Singapore
Description: Triple-A, the Digital Currency Payment Institution, enables businesses to pay and get
paid in both traditional and digital currencies, volatility-free.

Their mission is to build a more efficient global payment ecosystem, by bridging the gaps between traditional & digital currencies.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/triple-a-technologies-pte-ltd--1205740

Exhibitor Name: TripNinja
Address: B3J 3N2 Halifax, Canada
Description: Do you sell flights online? Trip Ninja 's SmartFlights API can help you increase your conversions and improve your revenue per search using your existing flights content.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/trip-ninja--1041791

Exhibitor Name: TRIPPYGOTOURSANDTRAVELLTD
Address: 0 NAIROBI, Kenya
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-li


Exhibitor Name: TTGTravelToGreece
Address: 17455 glyfada, Greece
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/ttg-travel-to-greece--1203508

Exhibitor Name: TT-TravelTimeGmbH
Address: 40721 Hilden, Germany
Description: 
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/tt-traveltime-gmbh--1041967

Exhibitor Name: TTX-TrafalgarTickets
Address: WC2R OLQ London, United Kingdom
Description: We connect people through creative, innovative experiences by providing inspiring entertainment for everyone for life.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibitor-list/#/detail/ttx-trafalgar-tickets--1206479

Exhibitor Name: TucayaTravel
Address: 110311 Bogotá, Colombia
Description: Our multicultural teams are here to meet all your expectations: tailor-made trips, honeymoon vacations, groups, corporate trips, or incentive programs.
href link: https://www.itb.com/en/itb-berlin-for-visitors/exhibito